Instalação das bibliotecas:


In [1]:
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
!pip install llama-index ipywidgets
!pip install pypdf transformers einops accelerate langchain bitsandbytes sentence-transformers
!pip install gradio
!pip install deep-translator

  Using cached huggingface_hub-0.25.2-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.25.2-py3-none-any.whl (436 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.5
    Uninstalling huggingface-hub-0.23.5:
      Successfully uninstalled huggingface-hub-0.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-huggingface 0.3.5 requires huggingface-hub<0.24.0,>=0.23.0, but you have huggingface-hub 0.25.2 which is incompatible.


In [2]:
!pip install --upgrade llama-index-llms-huggingface

  Using cached huggingface_hub-0.23.5-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.23.5-py3-none-any.whl (402 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.25.2
    Uninstalling huggingface-hub-0.25.2:
      Successfully uninstalled huggingface-hub-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.1.0 requires huggingface-hub>=0.25.1, but you have huggingface-hub 0.23.5 which is incompatible.


Login na Hugging Face:

*Necessário para acessar os modelos de linguagem hospedados no Hugging Face.*

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

Importação de bibliotecas:

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import gradio as gr
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

LLAMA2_13B_CHAT é o nome do modelo que será utilizado. O system_prompt define as instruções que guiarão o comportamento do assistente virtual, especificando que ele deve responder em português e fornecer explicações detalhadas.

In [5]:
# Configurar o modelo e tokenizer
LLAMA2_13B_CHAT = "meta-llama/Llama-2-13b-chat-hf"
system_prompt = """
Você é um assistente virtual especializado em ajudar estudantes com perguntas sobre diversos tópicos acadêmicos, incluindo negócios, tecnologia, biologia, ciências, e matemática. Responda SEMPRE em português, independentemente da pergunta ou do contexto, de forma clara e concisa. Forneça explicações detalhadas e exemplos práticos, se necessário.

Se uma pergunta não fizer sentido ou não for factualmente coerente, explique o porquê de forma educada, ao invés de responder algo incorreto. Se você não souber a resposta para uma pergunta, não compartilhe informações falsas.

Para perguntas acadêmicas, forneça explicações completas. Não forneça recomendações de coisas que não existem.

Seja sempre útil, respeitoso, e educado em suas respostas. Caso um aluno seja desrespeitoso, responda de forma educada.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + system_prompt + "<</SYS>>\n\n{query_str}[/INST] "
)

# Carregar o modelo e tokenizer
tokenizer = AutoTokenizer.from_pretrained(LLAMA2_13B_CHAT)
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(
    LLAMA2_13B_CHAT,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Configurar LLM
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=500,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=LLAMA2_13B_CHAT,
    model_name=LLAMA2_13B_CHAT,
    model=model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
Settings.llm = llm  # Configurar o LLM
Settings.embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")  # Configurar o modelo de embeddings

# Carregar documentos
documents = SimpleDirectoryReader("materiais").load_data()

# Criar o índice
index = VectorStoreIndex.from_documents(documents)

# Criar mecanismo de consulta
query_engine = index.as_query_engine()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from langdetect import detect
from deep_translator import GoogleTranslator

# Instanciar o tradutor
translator = GoogleTranslator(source='auto', target='pt')

# Função para verificar e traduzir para português
def translate_to_portuguese(text):
    detected_lang = detect(text)
    if detected_lang != 'pt':
        # Traduzir para o português
        translated = translator.translate(text)
        return translated
    return text

Interface Gradio:

In [9]:
# Função para consultar o `query_engine` e gerar uma resposta
def chatbot_response(message, history):
    response = query_engine.query(message)

    # Garantir que a resposta esteja em português
    response_in_portuguese = translate_to_portuguese(str(response))
    return response_in_portuguese

# Criar a interface Gradio no estilo de um chatbot
chatbot = gr.ChatInterface(fn=chatbot_response,
                           title="Assistente Virtual",
                           description="Converse com o chatbot e receba respostas para suas perguntas!",
                           theme="compact")

# Iniciar a interface Gradio com compartilhamento
if __name__ == "__main__":
    chatbot.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1025: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-67114acc-53fdf8a4550d3f424c20a987;924bc895-8899-40d6-99a7-729437e72dac)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://212a451f28ddb7dcf9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
